In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os

In [3]:
blue_shades = [
    "#1f4fd8",
    "#5779c9",
    "#144BB8",
    "#0055ff",
]
green_shades = [
    "#06d945",
    "#1da81d",
    "#1fc93e",
    "#1EA619",
]
orange_shades = [
    "#ff7f0e",
    "#ff9f3f",
    "#ff6f0e",
    "#ff5a00",
]
red_shade = "#ff0044"

vis_dict = {
    'M1': {'lbl':'M1, 2.5 um w/ WS2', 'color':red_shade},
    'M2': {'lbl':'M2, 0.5 um', 'color':green_shades[0]},
    'M3': {'lbl':'M3, 0.5 um', 'color':green_shades[1]},
    'M4': {'lbl':'M4, 0.5 um', 'color':green_shades[2]},
    'M5': {'lbl':'M5, 0.5 um', 'color':green_shades[3]},
    'M6': {'lbl':'M6, 1.0 um', 'color':blue_shades[0]},
    'M7': {'lbl':'M7, 1.0 um', 'color':blue_shades[1]},
    'M8': {'lbl':'M8, 1.0 um', 'color':blue_shades[2]},
    'M9': {'lbl':'M9, 1.0 um', 'color':blue_shades[3]},
    'M10': {'lbl':'M10, 1.5 um', 'color':orange_shades[0]},
    'M11': {'lbl':'M11, 1.5 um', 'color':orange_shades[1]}    

}

delimiter = '\t'  

In [47]:
'''
I have an external series resistor as a guardrail. 
'''
def account_for_resistor(df, R_series=0.991e6): #in ohms
    df['Vsample_V_crct'] = df['Vsample_V'] - df['Vsample_I'] * R_series
    return df

In [48]:
def build_dfs(directory):
    files = os.listdir(directory)
    dfs= {}
    labels = []
    dfs_to_merge = {}

    for file in files:
        if file.endswith('.dat'):
            label = file.split('_')[2].replace('.dat', '')
            if label not in labels:
                labels.append(label)
                dfs_to_merge[label] = []

    for file in files:
        if file.endswith('.dat'):
            label = file.split('_')[2].replace('.dat', '')
            df = pd.read_csv(os.path.join(directory, file), sep=delimiter)
            df = account_for_resistor(df)
            dfs_to_merge[label].append(df)

    for label in labels:
        df_list = dfs_to_merge[label]
        df = pd.concat(df_list, axis=0)
        df.index = range(len(df))
        dfs[label] = df

    labels = sorted(labels, key=lambda x: int(x[1:]))

    return dfs, labels

In [49]:
def plotter(fig, dfs, labels, device):
    for label in labels[::-1]:
        df = dfs[label]
        fig.add_trace(
            go.Scatter(
                x=df['Vsample_V_crct'],
                y=df['Vsample_I'],
                mode='lines',
                name=f'{vis_dict[label]["lbl"]}',
                line=dict(width=2, color=vis_dict[label]['color'])
            )
        )
    fig.update_layout(
    xaxis_title='V, V',
    yaxis_title='I, A',
    title=device,
    width=1000,
    height=800
    )
    fig.show()

# MS2P_6

In [50]:
device = "MS2P_6"
directory = '/Users/goikhmanb/Desktop/MS2P_6 Tunnel Data 2026_1_15'
dfs, labels = build_dfs(directory)

In [51]:
fig = go.Figure()
plotter(fig, dfs, labels, device)

In [52]:
fig.write_html(f"{device}_iv.html")

# MS2P_5
## 2 reps

In [53]:
device = "MS2P_5"
directory = "/Users/goikhmanb/Desktop/MS2P_5 Tunnel Data 2026_1_15/2reps"


In [54]:
dfs, labels = build_dfs(directory)
fig = go.Figure()
plotter(fig, dfs, labels, device)

In [55]:
fig.write_html(f"{device}_iv.html")

## 6reps

In [56]:
device = "MS2P_5"
directory = "/Users/goikhmanb/Desktop/MS2P_5 Tunnel Data 2026_1_15/6reps"

In [57]:
dfs, labels = build_dfs(directory)
fig = go.Figure()
plotter(fig, dfs, labels, device)

In [58]:
fig.write_html(f"{device}_iv_detailed.html")

In [59]:
directory =  '/Users/goikhmanb/Desktop/MS2P_6 Tunnel Data 2026_1_15'
files = os.listdir(directory)
files_M10 = [file for file in files if 'M10' in file and file.endswith('.dat')]
dfs_M10 = {}
for file in files_M10:
    label = file
    df = pd.read_csv(os.path.join(directory, file), sep=delimiter)
    dfs_M10[label] = df
files_M10

['S2_to_M10_4.dat', 'S2_to_M10_5.dat', 'S2_to_M10_3.dat']

In [46]:
fig = go.Figure()
for label in dfs_M10.keys():
    df = dfs_M10[label]
    fig.add_trace(
        go.Scatter(
            x=df['Vsample_V'],
            y=df['Vsample_I'],
            mode='lines',
            name=label,
            line=dict(width=2)
        )
    )
fig.update_layout(
xaxis_title='V, V',
yaxis_title='I, A',
title=device,
width=800,
height=500
)
fig.show()